In [3]:
from email.header import Header
from operator import index
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
import torch
import random
import scipy.spatial
import scipy.io

In [16]:
torch.cuda.is_available()

True

In [40]:
def get_strided_data_clust(dt, gt_size, horizon, step):
    inp_te = []
    dtt = dt.astype(np.float32)
    raw_data = dtt

    ped = raw_data.ped.unique()
    # print(ped)
    frame = []
    ped_ids = []
    # print(dt)
    # print(gt_size)
    # print(horizon)
    # print(step)
    for p in ped:
        # print(f"p is  {p}")
        # print("u p in ped")
        # print(raw_data[raw_data.ped == p].shape)

        # print(1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step)

        for i in range(
            1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step
        ):

            frame.append(
                dt[dt.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, [0]]
                .values.squeeze()
            )
            
            # print("%i,%i,%i" % (i * 4, i * 4 + gt_size, i * 4 + gt_size + horizon))
            inp_te.append(
                raw_data[raw_data.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, 2:6]
                .values
            )
            ped_ids.append(p)

    frames = np.stack(frame)
    inp_te_np = np.stack(inp_te)
    ped_ids = np.stack(ped_ids)

    # inp_relative_pos= inp_te_np-inp_te_np[:,:1,:]
    inp_speed = np.concatenate(
        (
            np.zeros((inp_te_np.shape[0], 1, 2)),
            inp_te_np[:, 1:, 0:2] - inp_te_np[:, :-1, 0:2],
        ),
        1,
    )
    # inp_accel = np.concatenate((np.zeros((inp_te_np.shape[0],1,2)),inp_speed[:,1:,0:2] - inp_speed[:, :-1, 0:2]),1)
    # inp_std = inp_no_start.std(axis=(0, 1))
    # inp_mean = inp_no_start.mean(axis=(0, 1))
    # inp_norm= inp_no_start
    # inp_norm = (inp_no_start - inp_mean) / inp_std

    # vis=inp_te_np[:,1:,2:4]/np.linalg.norm(inp_te_np[:,1:,2:4],2,axis=2)[:,:,np.newaxis]
    # inp_norm=np.concatenate((inp_norm,vis),2)
    inp_norm = np.concatenate((inp_te_np, inp_speed), 2)
    inp_mean = np.zeros(4)
    inp_std = np.ones(4)

    return (
        inp_norm[:, :gt_size],
        inp_norm[:, gt_size:],
        {
            "mean": inp_mean,
            "std": inp_std,
            "seq_start": inp_te_np[:, 0:1, :].copy(),
            "frames": frames,
            "peds": ped_ids,
        },
    )

In [42]:
def create_dataset(
    dataset_folder,
    dataset_name,
    val_size,
    gt,
    horizon,
    delim="\t",
    train=True,
    eval=False,
    verbose=False,
):

    if train == True:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "train"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "train")
    if train == False and eval == False:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "val"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "val")
    if train == False and eval == True:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "test"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "test")

    datasets_list = datasets_list
    print(datasets_list)
    data = {}
    data_src = []
    data_trg = []
    data_seq_start = []
    data_frames = []
    data_dt = []
    data_peds = []

    val_src = []
    val_trg = []
    val_seq_start = []
    val_frames = []
    val_dt = []
    val_peds = []
    print("ovdje")
    if verbose:
        print("start loading dataset")
        print("validation set size -> %i" % (val_size))

    for i_dt, dt in enumerate(datasets_list):
        if verbose:
            print("%03i / %03i - loading %s" % (i_dt + 1, len(datasets_list), dt))
        # if i_dt == 0 or i_dt == 1:
        # data = pd.read_csv(os.path.join(full_dt_folder, dt), sep="\t", header=None)
        # print(data.info())
        # print(data.head())
        raw_data = pd.read_csv(
            os.path.join(full_dt_folder, dt),
            delimiter=delim,
            names=["frame", "ped", "x", "y", "test_1", "test_2"],
            usecols=[0, 1, 2, 3, 4, 5],
            na_values="?",
        )

        #print(raw_data.head())
        raw_data.sort_values(by=["frame", "ped"], inplace=True)
        
        inp, out, info = get_strided_data_clust(raw_data, gt, horizon, 1)
        #print(inp, out, info)
        dt_frames = info["frames"]
        dt_seq_start = info["seq_start"]
        dt_dataset = np.array([i_dt]).repeat(inp.shape[0])
        dt_peds = info["peds"]
        print(inp)

        if val_size > 0 and inp.shape[0] > val_size * 2.5:
            if verbose:
                print("created validation from %s" % (dt))
            k = random.sample(np.arange(inp.shape[0]).tolist(), val_size)
            val_src.append(inp[k, :, :])
            val_trg.append(out[k, :, :])
            val_seq_start.append(dt_seq_start[k, :, :])
            val_frames.append(dt_frames[k, :])
            val_dt.append(dt_dataset[k])
            val_peds.append(dt_peds[k])
            inp = np.delete(inp, k, 0)
            out = np.delete(out, k, 0)
            dt_frames = np.delete(dt_frames, k, 0)
            dt_seq_start = np.delete(dt_seq_start, k, 0)
            dt_dataset = np.delete(dt_dataset, k, 0)
            dt_peds = np.delete(dt_peds, k, 0)
        elif val_size > 0:
            if verbose:
                print(
                    "could not create validation from %s, size -> %i"
                    % (dt, inp.shape[0])
                )

        data_src.append(inp)
        data_trg.append(out)
        data_seq_start.append(dt_seq_start)
        data_frames.append(dt_frames)
        data_dt.append(dt_dataset)
        data_peds.append(dt_peds)

    data["src"] = np.concatenate(data_src, 0)
    data["trg"] = np.concatenate(data_trg, 0)
    data["seq_start"] = np.concatenate(data_seq_start, 0)
    data["frames"] = np.concatenate(data_frames, 0)
    data["dataset"] = np.concatenate(data_dt, 0)
    data["peds"] = np.concatenate(data_peds, 0)
    data["dataset_name"] = datasets_list

    mean = data["src"].mean((0, 1))
    std = data["src"].std((0, 1))

    if val_size > 0:
        data_val = {}
        data_val["src"] = np.concatenate(val_src, 0)
        data_val["trg"] = np.concatenate(val_trg, 0)
        data_val["seq_start"] = np.concatenate(val_seq_start, 0)
        data_val["frames"] = np.concatenate(val_frames, 0)
        data_val["dataset"] = np.concatenate(val_dt, 0)
        data_val["peds"] = np.concatenate(val_peds, 0)

        return IndividualTfDataset(data, "train", mean, std), IndividualTfDataset(
            data_val, "validation", mean, std
        )

    return IndividualTfDataset(data, "train", mean, std), None

In [43]:
a, _ = create_dataset(
            "datasets",
            "fpl",
            val_size=0,
            gt=8,
            horizon=12,
            delim="\t",
            train=True,
            verbose=False
        )

['GOPR0236U20.txt']
ovdje
[[[ 7.15606506e+02  6.16524475e+02  1.00000001e-01  1.00000001e-01
    0.00000000e+00  0.00000000e+00]
  [ 7.31325012e+02  6.26952026e+02  1.09999999e-01  1.09999999e-01
    1.57185059e+01  1.04275513e+01]
  [ 7.46811523e+02  6.29562500e+02  1.19999997e-01  1.19999997e-01
    1.54865112e+01  2.61047363e+00]
  ...
  [ 7.22078003e+02  6.49086487e+02  1.40000001e-01  1.40000001e-01
   -1.04660034e+01  1.28100586e+00]
  [ 7.14283508e+02  6.51716980e+02  1.50000006e-01  1.50000006e-01
   -7.79449463e+00  2.63049316e+00]
  [ 7.15597473e+02  6.57002502e+02  1.59999996e-01  1.59999996e-01
    1.31396484e+00  5.28552246e+00]]

 [[ 7.31325012e+02  6.26952026e+02  1.09999999e-01  1.09999999e-01
    0.00000000e+00  0.00000000e+00]
  [ 7.46811523e+02  6.29562500e+02  1.19999997e-01  1.19999997e-01
    1.54865112e+01  2.61047363e+00]
  [ 7.45564514e+02  6.46489014e+02  1.19999997e-01  1.19999997e-01
   -1.24700928e+00  1.69265137e+01]
  ...
  [ 7.14283508e+02  6.51716980e+0

In [44]:
a[0]

{'src': tensor([[ 7.1561e+02,  6.1652e+02,  1.0000e-01,  1.0000e-01,  0.0000e+00,
           0.0000e+00],
         [ 7.3133e+02,  6.2695e+02,  1.1000e-01,  1.1000e-01,  1.5719e+01,
           1.0428e+01],
         [ 7.4681e+02,  6.2956e+02,  1.2000e-01,  1.2000e-01,  1.5487e+01,
           2.6105e+00],
         [ 7.4556e+02,  6.4649e+02,  1.2000e-01,  1.2000e-01, -1.2470e+00,
           1.6927e+01],
         [ 7.3254e+02,  6.4781e+02,  1.3000e-01,  1.3000e-01, -1.3021e+01,
           1.3165e+00],
         [ 7.2208e+02,  6.4909e+02,  1.4000e-01,  1.4000e-01, -1.0466e+01,
           1.2810e+00],
         [ 7.1428e+02,  6.5172e+02,  1.5000e-01,  1.5000e-01, -7.7945e+00,
           2.6305e+00],
         [ 7.1560e+02,  6.5700e+02,  1.6000e-01,  1.6000e-01,  1.3140e+00,
           5.2855e+00]]),
 'trg': tensor([[ 7.1692e+02,  6.7389e+02,  1.7000e-01,  1.7000e-01,  1.3255e+00,
           1.6891e+01],
         [ 7.2869e+02,  6.8298e+02,  1.8000e-01,  1.8000e-01,  1.1768e+01,
           9.0875e

In [5]:
class IndividualTfDataset(Dataset):
    def __init__(self, data, name, mean, std):
        super(IndividualTfDataset, self).__init__()

        self.data = data
        self.name = name

        self.mean = mean
        self.std = std

    def __len__(self):
        return self.data["src"].shape[0]

    def __getitem__(self, index):
        return {
            "src": torch.Tensor(self.data["src"][index]),
            "trg": torch.Tensor(self.data["trg"][index]),
            "frames": self.data["frames"][index],
            "seq_start": self.data["seq_start"][index],
            "dataset": self.data["dataset"][index],
            "peds": self.data["peds"][index],
        }

In [6]:
def get_strided_data(dt, gt_size, horizon, step):
    inp_te = []
    dtt = dt.astype(np.float32)
    raw_data = dtt

    ped = raw_data.ped.unique()
    frame = []
    ped_ids = []
    for p in ped:
        for i in range(
            1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step
        ):
            frame.append(
                dt[dt.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, [0]]
                .values.squeeze()
            )
            # print("%i,%i,%i" % (i * 4, i * 4 + gt_size, i * 4 + gt_size + horizon))
            inp_te.append(
                raw_data[raw_data.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, 2:4]
                .values
            )
            ped_ids.append(p)

    frames = np.stack(frame)
    inp_te_np = np.stack(inp_te)
    ped_ids = np.stack(ped_ids)

    inp_no_start = inp_te_np[:, 1:, 0:2] - inp_te_np[:, :-1, 0:2]
    inp_std = inp_no_start.std(axis=(0, 1))
    inp_mean = inp_no_start.mean(axis=(0, 1))
    inp_norm = inp_no_start
    # inp_norm = (inp_no_start - inp_mean) / inp_std

    # vis=inp_te_np[:,1:,2:4]/np.linalg.norm(inp_te_np[:,1:,2:4],2,axis=2)[:,:,np.newaxis]
    # inp_norm=np.concatenate((inp_norm,vis),2)

    return (
        inp_norm[:, : gt_size - 1],
        inp_norm[:, gt_size - 1 :],
        {
            "mean": inp_mean,
            "std": inp_std,
            "seq_start": inp_te_np[:, 0:1, :].copy(),
            "frames": frames,
            "peds": ped_ids,
        },
    )

In [7]:
def get_strided_data_2(dt, gt_size, horizon, step):
    inp_te = []
    dtt = dt.astype(np.float32)
    raw_data = dtt

    ped = raw_data.ped.unique()
    frame = []
    ped_ids = []
    for p in ped:
        for i in range(
            1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step
        ):
            frame.append(
                dt[dt.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, [0]]
                .values.squeeze()
            )
            # print("%i,%i,%i" % (i * 4, i * 4 + gt_size, i * 4 + gt_size + horizon))
            inp_te.append(
                raw_data[raw_data.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, 2:4]
                .values
            )
            ped_ids.append(p)

    frames = np.stack(frame)
    inp_te_np = np.stack(inp_te)
    ped_ids = np.stack(ped_ids)

    inp_relative_pos = inp_te_np - inp_te_np[:, :1, :]
    inp_speed = np.concatenate(
        (
            np.zeros((inp_te_np.shape[0], 1, 2)),
            inp_te_np[:, 1:, 0:2] - inp_te_np[:, :-1, 0:2],
        ),
        1,
    )
    inp_accel = np.concatenate(
        (
            np.zeros((inp_te_np.shape[0], 1, 2)),
            inp_speed[:, 1:, 0:2] - inp_speed[:, :-1, 0:2],
        ),
        1,
    )
    # inp_std = inp_no_start.std(axis=(0, 1))
    # inp_mean = inp_no_start.mean(axis=(0, 1))
    # inp_norm= inp_no_start
    # inp_norm = (inp_no_start - inp_mean) / inp_std

    # vis=inp_te_np[:,1:,2:4]/np.linalg.norm(inp_te_np[:,1:,2:4],2,axis=2)[:,:,np.newaxis]
    # inp_norm=np.concatenate((inp_norm,vis),2)
    inp_norm = np.concatenate((inp_te_np, inp_relative_pos, inp_speed, inp_accel), 2)
    inp_mean = np.zeros(8)
    inp_std = np.ones(8)

    return (
        inp_norm[:, :gt_size],
        inp_norm[:, gt_size:],
        {
            "mean": inp_mean,
            "std": inp_std,
            "seq_start": inp_te_np[:, 0:1, :].copy(),
            "frames": frames,
            "peds": ped_ids,
        },
    )

In [8]:
def distance_metrics(gt, preds):
    errors = np.zeros(preds.shape[:-1])
    for i in range(errors.shape[0]):
        for j in range(errors.shape[1]):
            errors[i, j] = scipy.spatial.distance.euclidean(gt[i, j], preds[i, j])
    return errors.mean(), errors[:, -1].mean(), errors

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformer.decoder import Decoder
from transformer.multihead_attention import MultiHeadAttention
from transformer.positional_encoding import PositionalEncoding
from transformer.pointerwise_feedforward import PointerwiseFeedforward
from transformer.encoder_decoder import EncoderDecoder
from transformer.encoder import Encoder
from transformer.encoder_layer import EncoderLayer
from transformer.decoder_layer import DecoderLayer
from transformer.batch import subsequent_mask
import numpy as np
import scipy.io
import os

import copy
import math

class IndividualTF(nn.Module):
    def __init__(self, enc_inp_size, dec_inp_size, dec_out_size, N=6,
                   d_model=512, d_ff=2048, h=8, dropout=0.1,mean=[0,0],std=[0,0]):
        super(IndividualTF, self).__init__()
        "Helper: Construct a model from hyperparameters."
        c = copy.deepcopy
        attn = MultiHeadAttention(h, d_model)
        ff = PointerwiseFeedforward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        self.mean=np.array(mean)
        self.std=np.array(std)
        self.model = EncoderDecoder(
            Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
            Decoder(DecoderLayer(d_model, c(attn), c(attn),
                                 c(ff), dropout), N),
            nn.Sequential(LinearEmbedding(enc_inp_size,d_model), c(position)),
            nn.Sequential(LinearEmbedding(dec_inp_size,d_model), c(position)),
            Generator(d_model, dec_out_size))

        # This was important from their code.
        # Initialize parameters with Glorot / fan_avg.
        for p in self.model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)





    def forward(self, *input):
        return self.model.generator(self.model(*input))

class LinearEmbedding(nn.Module):
    def __init__(self, inp_size,d_model):
        super(LinearEmbedding, self).__init__()
        # lut => lookup table
        self.lut = nn.Linear(inp_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, out_size):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, out_size)

    def forward(self, x):
        return self.proj(x)

In [20]:
import argparse
import baselineUtils
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import os
import time
from transformer.batch import subsequent_mask
from torch.optim import Adam, SGD, RMSprop, Adagrad
from transformer.noam_opt import NoamOpt
import numpy as np
import scipy.io
import json
import pickle

from torch.utils.tensorboard import SummaryWriter
import individual_TF

train_dataset, _ = create_dataset(
            "datasets",
            "fpl",
            val_size=0,
            gt=8,
            horizon=12,
            delim="\t",
            train=True,
            verbose=False
        )
val_dataset, _ = create_dataset(
            "datasets",
            "fpl",
            val_size=0,
            gt=8,
            horizon=12,
            delim="\t",
            train=False,
            verbose=False)

   

model = individual_TF.IndividualTF(
        2,
        3,
        3,
        N=6,
        d_model=512,
        d_ff=2048,
        h=8,
        dropout=0.1,
        mean=[0, 0],
        std=[0, 0],
    ).to(device)
    

tr_dl = torch.utils.data.DataLoader(
        train_dataset, batch_size=70, shuffle=True, num_workers=0
    )

    
val_dl = torch.utils.data.DataLoader(
        val_dataset, batch_size=70, shuffle=True, num_workers=0
    )

test_dl = torch.utils.data.DataLoader(
        test_dataset, batch_size=70, shuffle=False, num_workers=0
    )

    # optim = SGD(list(a.parameters())+list(model.parameters())+list(generator.parameters()),lr=0.01)
    # sched=torch.optim.lr_scheduler.StepLR(optim,0.0005)
optim = NoamOpt(
        512,
        1,
        len(tr_dl) * 10,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9),
    )
    # optim=Adagrad(list(a.parameters())+list(model.parameters())+list(generator.parameters()),lr=0.01,lr_decay=0.001)
epoch = 0

    # mean=train_dataset[:]['src'][:,1:,2:4].mean((0,1))
mean = torch.cat(
        (train_dataset[:]["src"][:, 1:, 2:4], train_dataset[:]["trg"][:, :, 2:4]), 1
    ).mean((0, 1))
    # std=train_dataset[:]['src'][:,1:,2:4].std((0,1))
std = torch.cat(
    (train_dataset[:]["src"][:, 1:, 2:4], train_dataset[:]["trg"][:, :, 2:4]), 1
).std((0, 1))
    
means = []
stds = []
for i in np.unique(train_dataset[:]["dataset"]):
    ind = train_dataset[:]["dataset"] == i
    means.append(
        torch.cat(
            (
                train_dataset[:]["src"][ind, 1:, 2:4],
                train_dataset[:]["trg"][ind, :, 2:4],
            ),
            1,
        ).mean((0, 1))
    )
    stds.append(
        torch.cat(
                (
                    train_dataset[:]["src"][ind, 1:, 2:4],
                    train_dataset[:]["trg"][ind, :, 2:4],
                ),
                1,
            ).std((0, 1))
        )
mean = torch.stack(means).mean(0)
std = torch.stack(stds).mean(0)


max_epoch = 5
while epoch < max_epoch:
    epoch_loss = 0
    model.train()

    for id_b, batch in enumerate(tr_dl):

        optim.optimizer.zero_grad()
        inp = (batch["src"][:, 1:, 2:4].to(device) - mean.to(device)) / std.to(
            device
        )
        target = (batch["trg"][:, :-1, 2:4].to(device) - mean.to(device)) / std.to(
            device
        )
        target_c = torch.zeros((target.shape[0], target.shape[1], 1)).to(device)
        target = torch.cat((target, target_c), -1)
        start_of_seq = (
            torch.Tensor([0, 0, 1])
            .unsqueeze(0)
            .unsqueeze(1)
            .repeat(target.shape[0], 1, 1)
            .to(device)
        )

        dec_inp = torch.cat((start_of_seq, target), 1)

        src_att = torch.ones((inp.shape[0], 1, inp.shape[1])).to(device)
        trg_att = (
            subsequent_mask(dec_inp.shape[1])
            .repeat(dec_inp.shape[0], 1, 1)
            .to(device)
        )

        pred = model(inp, dec_inp, src_att, trg_att)

        loss = (
            F.pairwise_distance(
                pred[:, :, 0:2].contiguous().view(-1, 2),
                (
                    (batch["trg"][:, :, 2:4].to(device) - mean.to(device))
                    / std.to(device)
                )
                .contiguous()
                .view(-1, 2)
                .to(device),
            ).mean()
            + torch.mean(torch.abs(pred[:, :, 2]))
        )
        loss.backward()
        optim.step()
        
        epoch_loss += loss.item()
        # sched.step()
    log.add_scalar("Loss/train", epoch_loss / len(tr_dl), epoch)
    with torch.no_grad():
        model.eval()

        val_loss = 0
        step = 0
        model.eval()
        gt = []
        pr = []
        inp_ = []
        peds = []
        frames = []
        dt = []

        for id_b, batch in enumerate(val_dl):
            inp_.append(batch["src"])
            gt.append(batch["trg"][:, :, 0:2])
            frames.append(batch["frames"])
            peds.append(batch["peds"])
            dt.append(batch["dataset"])

            inp = (batch["src"][:, 1:, 2:4].to(device) - mean.to(device)) / std.to(
                    device
            )
            src_att = torch.ones((inp.shape[0], 1, inp.shape[1])).to(device)
            start_of_seq = (
                torch.Tensor([0, 0, 1])
                .unsqueeze(0)
                .unsqueeze(1)
                .repeat(inp.shape[0], 1, 1)
                .to(device)
            )
            dec_inp = start_of_seq
            
            #preds
            for i in range(12):
                trg_att = (
                    subsequent_mask(dec_inp.shape[1])
                    .repeat(dec_inp.shape[0], 1, 1)
                    .to(device)
                )
                out = model(inp, dec_inp, src_att, trg_att)
                dec_inp = torch.cat((dec_inp, out[:, -1:, :]), 1)

            preds_tr_b = (
                dec_inp[:, 1:, 0:2] * std.to(device) + mean.to(device)
            ).cpu().numpy().cumsum(1) + batch["src"][:, -1:, 0:2].cpu().numpy()
            pr.append(preds_tr_b)
            

        peds = np.concatenate(peds, 0)
        frames = np.concatenate(frames, 0)
        dt = np.concatenate(dt, 0)
        gt = np.concatenate(gt, 0)
        dt_names = test_dataset.data["dataset_name"]
        pr = np.concatenate(pr, 0)
        mad, fad, errs = baselineUtils.distance_metrics(gt, pr)
        log.add_scalar("validation/MAD", mad, epoch)
        log.add_scalar("validation/FAD", fad, epoch)

        if True:

            model.eval()
            gt = []
            pr = []
            inp_ = []
            peds = []
            frames = []
            dt = []

            for id_b, batch in enumerate(test_dl):
                inp_.append(batch["src"])
                gt.append(batch["trg"][:, :, 0:2])
                frames.append(batch["frames"])
                peds.append(batch["peds"])
                dt.append(batch["dataset"])

                inp = (
                    batch["src"][:, 1:, 2:4].to(device) - mean.to(device)
                ) / std.to(device)
                src_att = torch.ones((inp.shape[0], 1, inp.shape[1])).to(device)
                start_of_seq = (
                    torch.Tensor([0, 0, 1])
                    .unsqueeze(0)
                    .unsqueeze(1)
                    .repeat(inp.shape[0], 1, 1)
                    .to(device)
                )
                dec_inp = start_of_seq
                #preds
                for i in range(12):
                    trg_att = (
                        subsequent_mask(dec_inp.shape[1])
                        .repeat(dec_inp.shape[0], 1, 1)
                        .to(device)
                    )
                    out = model(inp, dec_inp, src_att, trg_att)
                    dec_inp = torch.cat((dec_inp, out[:, -1:, :]), 1)

                preds_tr_b = (
                    dec_inp[:, 1:, 0:2] * std.to(device) + mean.to(device)
                ).cpu().numpy().cumsum(1) + batch["src"][:, -1:, 0:2].cpu().numpy()
                pr.append(preds_tr_b)
                

            peds = np.concatenate(peds, 0)
            frames = np.concatenate(frames, 0)
            dt = np.concatenate(dt, 0)
            gt = np.concatenate(gt, 0)
            dt_names = test_dataset.data["dataset_name"]
            pr = np.concatenate(pr, 0)
            mad, fad, errs = baselineUtils.distance_metrics(gt, pr)

            

        
    epoch += 1
ab = 1

['GOPR0236U20.txt', 'GOPR0238U20.txt', 'GOPR0241U20.txt', 'GOPR0242U20.txt', 'GOPR0245U20.txt']
ovdje
   frame  ped         x         y
0      3    0  715.6065  616.5245
1      4    0  731.3250  626.9520
2      5    0  746.8115  629.5625
3      6    0  745.5645  646.4890
4      7    0  732.5440  647.8055
     frame  ped         x         y
0        3    0  715.6065  616.5245
332      3   10  551.4005  563.0890
379      3   20  800.3700  531.8260
1        4    0  731.3250  626.9520
333      4   10  567.0265  570.9215
   frame  ped         x         y
0     33   20  591.8165  464.0145
1     34   20  581.2315  460.1445
2     35   20  578.7150  470.6395
3     36   20  560.5225  482.2710
4     37   20  542.2580  470.6605
   frame  ped         x         y
0     33   20  591.8165  464.0145
1     34   20  581.2315  460.1445
2     35   20  578.7150  470.6395
3     36   20  560.5225  482.2710
4     37   20  542.2580  470.6605
   frame  ped         x         y
0      6  140  812.1210  651.6835
1 

NameError: name 'device' is not defined

In [25]:
a, _ = create_dataset(
            "datasets",
            "fpl",
            val_size=0,
            gt=8,
            horizon=12,
            delim="\t",
            train=True,
            verbose=False
        )

['GOPR0236U20.txt']
ovdje
   frame  ped         x         y
0      3    0  715.6065  616.5245
1      4    0  731.3250  626.9520
2      5    0  746.8115  629.5625
3      6    0  745.5645  646.4890
4      7    0  732.5440  647.8055
     frame  ped         x         y
0        3    0  715.6065  616.5245
332      3   10  551.4005  563.0890
379      3   20  800.3700  531.8260
1        4    0  731.3250  626.9520
333      4   10  567.0265  570.9215


In [26]:
a[0]

{'src': tensor([[715.6065, 616.5245,   0.0000,   0.0000],
         [731.3250, 626.9520,  15.7185,  10.4276],
         [746.8115, 629.5625,  15.4865,   2.6105],
         [745.5645, 646.4890,  -1.2470,  16.9265],
         [732.5440, 647.8055, -13.0205,   1.3165],
         [722.0780, 649.0865, -10.4660,   1.2810],
         [714.2835, 651.7170,  -7.7945,   2.6305],
         [715.5975, 657.0025,   1.3140,   5.2855]]),
 'trg': tensor([[ 7.1692e+02,  6.7389e+02,  1.3255e+00,  1.6891e+01],
         [ 7.2869e+02,  6.8298e+02,  1.1768e+01,  9.0875e+00],
         [ 7.3256e+02,  6.8298e+02,  3.8695e+00,  0.0000e+00],
         [ 7.3258e+02,  6.7653e+02,  1.6479e-02, -6.4530e+00],
         [ 7.2873e+02,  6.7518e+02, -3.8445e+00, -1.3440e+00],
         [ 7.2859e+02,  6.7518e+02, -1.4197e-01, -3.5400e-03],
         [ 7.2338e+02,  6.6738e+02, -5.2141e+00, -7.8030e+00],
         [ 7.1823e+02,  6.5692e+02, -5.1495e+00, -1.0458e+01],
         [ 7.2477e+02,  6.5041e+02,  6.5460e+00, -6.5050e+00],
         

In [27]:
a[1]

{'src': tensor([[731.3250, 626.9520,   0.0000,   0.0000],
         [746.8115, 629.5625,  15.4865,   2.6105],
         [745.5645, 646.4890,  -1.2470,  16.9265],
         [732.5440, 647.8055, -13.0205,   1.3165],
         [722.0780, 649.0865, -10.4660,   1.2810],
         [714.2835, 651.7170,  -7.7945,   2.6305],
         [715.5975, 657.0025,   1.3140,   5.2855],
         [716.9230, 673.8935,   1.3255,  16.8910]]),
 'trg': tensor([[ 7.2869e+02,  6.8298e+02,  1.1768e+01,  9.0875e+00],
         [ 7.3256e+02,  6.8298e+02,  3.8695e+00,  0.0000e+00],
         [ 7.3258e+02,  6.7653e+02,  1.6479e-02, -6.4530e+00],
         [ 7.2873e+02,  6.7518e+02, -3.8445e+00, -1.3440e+00],
         [ 7.2859e+02,  6.7518e+02, -1.4197e-01, -3.5400e-03],
         [ 7.2338e+02,  6.6738e+02, -5.2141e+00, -7.8030e+00],
         [ 7.1823e+02,  6.5692e+02, -5.1495e+00, -1.0458e+01],
         [ 7.2477e+02,  6.5041e+02,  6.5460e+00, -6.5050e+00],
         [ 7.3520e+02,  6.3744e+02,  1.0428e+01, -1.2974e+01],
         